Copyright 2024 Google LLC. Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License

# Forecasting county-level unemployment rates using TimesFM and PDFM embeddings.

Note: This notebook demonstrates a highly experimental approach for forecasting county-level unemployment rates using TimesFM and PDFM embeddings. The methods and models applied here are in the early stages of development and may not yet provide reliable, production-grade forecasts. Results should be interpreted with caution, and further validation and tuning are recommended before applying these models in real-world scenarios.

# Data Preparation

### Step 1: Download a csv file of the embeddings using this [link](https://docs.google.com/forms/d/e/1FAIpQLSeZLIqTCIx1-OiBzUnqXZpu_k5M223ZvMmqwQhMZ_0TkaWhEQ/viewform).

The county and ZCTA (zipcode census tabulation area) embeddings are available in different files.

Here we assume that you have obtained the embeddings and uploaded them to a Google Drive directory called `pdfm_embeddings/v0/us`.

In [ ]:
import pandas as pd
from google.colab import drive
#@markdown Specify the path to the embeddings file.
embeddings_file_path = '/content/drive/MyDrive/pdfm_embeddings/v0/us/county_embeddings.csv' #@param {type:"string"}

drive.mount('/content/drive', force_remount=True)
county_embeddings = pd.read_csv(embeddings_file_path).set_index('place')

In [ ]:
county_embeddings

In [ ]:
embedding_features = [f'feature{x}' for x in range(330)]
county_embeddings.head(2)

### Step 2: Download and prepare monthly unemployment data at county level from Data Commons

In [ ]:
!pip install datacommons_pandas --upgrade --quiet
import datacommons_pandas as dc

In [ ]:
label = 'UnemploymentRate_Person'
# Due to response size limits, we'll query in batches.
batch_size = 200

all_labels = []
for start in range(0, county_embeddings.index.shape[0], batch_size):
    batch_indices = county_embeddings.index[start : start + batch_size]
    batch_data = dc.build_time_series_dataframe(batch_indices, label)
    all_labels.append(batch_data)

df_labels = pd.concat(all_labels)

print(df_labels.shape)
df_labels.head(2)

In [ ]:
import matplotlib.dates as mdates
_ax = df_labels.loc['geoId/01001'].plot(figsize=(10, 3))

# Modeling



In [ ]:
#@title Install TimesFM

%%capture
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['JAX_PMAP_USE_TENSORSTORE'] = 'false'
!pip install -q timesfm
import timesfm

In [ ]:
#@title Initialize a TimesFM model

# Can change to "cpu" if GPUs are not available, but it would be much slower.
timesfm_backend = "gpu"
timesfm_model_config = timesfm.TimesFmHparams(
    context_len=512,
    horizon_len=128,
    per_core_batch_size=128,
    backend=timesfm_backend,
)
timesfm_model = timesfm.TimesFm(
    hparams=timesfm_model_config,
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id="google/timesfm-1.0-200m-pytorch"
    )
)

In [ ]:
#@title Define our forecasting model
import lightgbm as lgbm
import numpy as np

class PDTimesFM:
  """Forecasts time series data using TimesFM and PDFM embeddings for error correction.

  This class combines the strengths of TimesFM for base forecasting and PDFM
  embeddings
  for capturing spatial correlations to improve prediction accuracy.
  """

  def __init__(
      self, timesfm_model, pdfm_embeddings, error_correction_model=None
  ):
    """Initializes the PDTimesFM model with a TimesFM model for forecasting

    and PDFM embeddings for error correction.

    Args:
        timesfm_model: The TimesFM model to use for forecasting.
        pdfm_embeddings: A pandas DataFrame containing the PDFM embeddings.
        error_correction_model: An optional error correction model to use.
    """
    self.timesfm_model = timesfm_model
    self.pdfm_embeddings = pdfm_embeddings
    self.error_correction_model = error_correction_model or lgbm.LGBMRegressor(
        max_leaf_nodes=19,
        min_samples_leaf=5,
        learning_rate=0.05,
        n_estimators=400,
        feature_fraction=0.8,
        verbose=-1,
    )

  def get_timesfm_forecast(self, history, forecast_steps=24, batch_size=128):
    """Obtains TimesFM forecasts in batches to handle large datasets.

    Args:
        history: A pandas DataFrame containing the historical time series data.
        forecast_steps: The number of steps to forecast into the future.
        batch_size: The size of each batch for processing.

    Returns:
        A pandas DataFrame containing the TimesFM forecasts for all locations.
    """
    all_forecasts = []
    for i in range(0, history.shape[0], batch_size):
      batch_history = history.iloc[i : i + batch_size]
      mean_forecast, _ = self.timesfm_model.forecast(
          inputs=batch_history.values
      )
      all_forecasts.append(mean_forecast[:, :forecast_steps])

    return pd.DataFrame(
        data=np.concatenate(all_forecasts),
        index=history.index,
    )

  def predict(self, history, train_time_steps=3, forecast_steps=24):
    """Generates forecasts using the PDTimesFM model with error correction.

    Args:
        history: A pandas DataFrame containing the historical spatiotemporal
          data.
        train_time_steps: The number of recent time steps to use for training
          the error correction model.
        forecast_steps: The number of steps to forecast into the future.

    Returns:
        A pandas DataFrame containing the adjusted forecasts.
    """
    history = history.loc[
        history.index.intersection(self.pdfm_embeddings.index)
    ].copy()

    # Split history into base forecast and error correction parts
    if train_time_steps:
      history_for_timesfm = history.iloc[:, :-train_time_steps]
      history_for_error_correction = history.iloc[:, -train_time_steps:]
    else:
      history_for_timesfm = history
      history_for_error_correction = None

    timesfm_forecast_steps = train_time_steps + forecast_steps
    timesfm_forecast = self.get_timesfm_forecast(
        history_for_timesfm, timesfm_forecast_steps
    )

    if not train_time_steps:
      return timesfm_forecast

    # Prepare data for error correction
    embedding_features = [f'feature{x}' for x in range(330)]
    train_data = []
    for i in range(train_time_steps):
      label = history_for_error_correction.iloc[:, i]
      forecast = timesfm_forecast.iloc[:, i]
      train_data_i = (
          self.pdfm_embeddings[embedding_features]
          .join(forecast, how='inner')
          .join(label, rsuffix='_gt')
      )
      train_data_i.columns = embedding_features + ['forecast', 'label']
      train_data.append(train_data_i)
    train_data = pd.concat(train_data)

    # Train error correction model
    self.error_correction_model.fit(
        train_data[embedding_features + ['forecast']], train_data['label']
    )

    # Apply error correction to base forecasts
    adjusted_forecast = []
    for i in range(train_time_steps, timesfm_forecast_steps):
      forecast = timesfm_forecast.iloc[:, i]
      x = self.pdfm_embeddings[embedding_features].join(forecast, how='inner')
      y = self.error_correction_model.predict(x)
      adjusted_forecast.append(y)

    adjusted_forecast = pd.DataFrame(
        data=np.column_stack(adjusted_forecast),
        index=history.index,
    )
    return adjusted_forecast


In [ ]:
#@title Use 10 years of data until 2022-07 to forecast the next 24 months.
timesteps = df_labels.columns
history_steps = timesteps[(timesteps >= '2012-07') & (timesteps < '2022-07')]
forecast_steps = timesteps[(timesteps >= '2022-07') & (timesteps < '2024-07')]

history = df_labels[history_steps]
pdtfm = PDTimesFM(timesfm_model, county_embeddings)
pdtfm_forecast = pdtfm.predict(history, train_time_steps=3)

In [ ]:
#@title Get TimesFM forecasts without adjustments for comparisons
tfm_forecast = pdtfm.predict(history, train_time_steps=0)
tfm_forecast.head(2)

In [ ]:
#@title Evaluate both sets of monthly predictions

from sklearn import metrics
import numpy as np
def evaluate(y_true, y_pred):
  return {
      'MAE': round(metrics.mean_absolute_error(y_true, y_pred), 3),
      'MAPE': round(metrics.mean_absolute_percentage_error(y_true, y_pred), 3),
      'R2': round(metrics.r2_score(y_true, y_pred), 2),
  }

all_metrics = []
for i, step in enumerate(forecast_steps):
  gt = df_labels[step]
  tfm_metrics = evaluate(gt, tfm_forecast.iloc[:, i])
  tfm_metrics['model'] = 'TimesFM'
  tfm_metrics['step'] = step
  all_metrics.append(tfm_metrics)
  pdt_metrics = evaluate(gt, pdtfm_forecast.iloc[:, i])
  pdt_metrics['model'] = 'PDTimesFM'
  pdt_metrics['step'] = step
  all_metrics.append(pdt_metrics)
  print('\n====', step)
  print('TimesFM', tfm_metrics)
  print('PDTimesFM', pdt_metrics)

all_metrics = pd.DataFrame(all_metrics)
all_metrics.groupby('model')[['MAE', 'MAPE', 'R2']].mean()

In [ ]:
#@title Plot metrics over time
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(3, 1, sharex=True)
sns.lineplot(data=all_metrics, x='step', y='MAPE', hue='model', ax=ax[0], legend=False)
sns.lineplot(data=all_metrics, x='step', y='MAE', hue='model', ax=ax[1], legend=False)
sns.lineplot(data=all_metrics, x='step', y='R2', hue='model', ax=ax[2])
ax[2].set(ylabel='$R^2$')
ax[2].legend(title='Model')
plt.suptitle('Forecasting errors and $R^2$ over time')
_ = plt.xticks(rotation=45)